In [15]:
import numpy as np
import pandas as pd
import os

import datasets
from datasets import load_dataset
from datasets import Dataset

In [14]:
# two installations required for hugging face implementation
from huggingface_hub import login
login()

In [16]:
# importing the csv file
df = pd.read_csv("data/bias_data.csv")

# convert to hugging face dataset
dataset = Dataset.from_pandas(df)

In [18]:
# splitting into testing and training datasets
train_test_split = dataset.train_test_split(test_size=0.2)

In [19]:
import datasets
from datasets import load_dataset
# we are going to pull from the repository specific to this project that contains the testing and training datasets

train_test_split["train"][100]

{'Unnamed: 0': 958,
 'attribute_in_window': True,
 'comment': 'i live here theres homeless white people in the area too. the only person saying black people are criminals is you.',
 'phrase': 'the area too. the only person saying black people are criminals is you.',
 'bias_sent': '0.0',
 'bias_phrase': 0.0,
 'bias_type': 'race'}

In [21]:
# using a tokenizer to process the text and include a padding and truncation strategy to handle any variable sequence
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/11377 [00:00<?, ? examples/s]


KeyError: 'text'